손으로 쓴 우편번호를 우체국에서 자동화하기 위한 만든 코드

In [10]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt

In [12]:
from tensorflow.examples.tutorials.mnist import input_data
# Check out https://www.tensorflow.org/get_started/mnist/beginners for more information about
# the mnist dataset

In [27]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True) 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


one_hot = True를 해놓으면 Y값을 one_hot으로 읽어온다는 뜻

In [15]:
nb_classes = 10

In [17]:
# MNIST data image of shape 28*28 = 784
X = tf.placeholder(tf.float32, [None,784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None,nb_classes])
W = tf.Variable(tf.random_normal([784,nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

In [18]:
# hypothesis using softmax
hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis),axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [24]:
# Test model
is_correct = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

In [28]:
# Parameters
training_epochs = 15
batch_size = 100

데이터가 크니까 잘라서 학습시킴<br>

전체 데이터셋을 한 번 학습시킨 것이 1 epoch이다.<br>
데이터셋이 너무 많으면 한번에 학습 시킬 수 없기때문에 batch size만큼씩 잘라서 진행한다<br>

### Training epoch / batch
In the neural network terminology: <br>

* one epoch = one forward pass and one backward pass of all the training examples<br>
* batch size = the number of training examples in of forward/backward pass. The higher the batch size, the more memory space you'll need.<br>
* number of iterations = number of passes, each pass using [batch size] number of examples. To be clear, one pass = one forward + one backward pass (we do not count the forward pass and backward pass as two different passes).<br>
<br>
Example: if you have 1000 training examples, and your batch size is 500, then it will take 2 iterations to complete 1 epoch.

In [36]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch): # i가 iteration
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c,_ = sess.run([cost,optimizer], feed_dict={X:batch_xs, Y:batch_ys})
            avg_cost += c / total_batch
            
        print ('Epoch: ','%04d' % (epoch+1),'cost= ','{:.9f}'.format(avg_cost))
        
        # Test the model using test sets
        print("Accuarcy: ", accuracy.eval(session=sess,
                                 feed_dict={X:mnist.test.images, Y:mnist.test.labels}))
        
        

Epoch:  0001 cost=  3.025242689
Accuarcy:  0.7438
Label: [2]
Prediction: [2]


AttributeError: 'DataSet' object has no attribute 'image'

accuracy.eval() = sess.run() 같은 기능을 한다.

# Sample image show and prediction

In [32]:
import matplotlib.pyplot as plt
import random

In [35]:
r = random.randint(0, mnist.test.num_examples - 1)
print("Label:", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prediction:", sess.run(tf.argmax(hypothesis,1),
                             feed_dict={X:mnist.test.images[r:r+1]}))
plt.imshow(mnist.test.image[r:r+1].reshape(28,28),cmap='Greys', interpolation = 'nearest')
plt.show()

RuntimeError: Attempted to use a closed Session.